# 8.8 구현하기 : 정규화 학장 클래스

## 8.8.1 기반 클래스 파일 실행시키기

In [6]:
%run ../chap07/cnn_basic_model.ipynb

## 8.8.2 CnnRegModel 클래스 선언

In [7]:
class CnnRegModel(CnnBasicModel):
    def __init__(self, name, dataset, hconfigs, show_maps=False,
                l2_decay=0, l1_decay=0):
        self.l2_decay = l2_decay
        self.l1_decay = l1_decay
        super(CnnRegModel, self).__init__(name, dataset, hconfigs, show_maps)
        
    def exec_all(self, epoch_count=10, batch_size=10, learning_rate=0.001,
                 report=0, show_cnt=3, show_params=False):
        super(CnnRegModel, self).exec_all(epoch_count, batch_size,
                                          learning_rate, report, show_cnt)
        if show_params: self.show_param_dist()          

## 8.8.3 파라미터 분포 출력 메서드 정의

In [8]:
def cnn_reg_show_param_dist(self):
    params = self.collect_params()
    mu = np.mean(params)
    sigma = np.sqrt(np.var(params))
    plt.hist(params, 100, density=True, facecolor='g', alpha=0.75)
    plt.axis([-0.2, 0.2, 0, 20.0])
    plt.text(0.08, 15.0, 'mu={:5.3f}'.format(mu))
    plt.text(0.08, 13.0, 'sigma={:5.3f}'.format(sigma))
    plt.grid(True)
    plt.show()
    
    total_count = len(params)
    near_zero_count = len(list(x for x in params if -1e-5 <= x <= 1e-5))
    print('Near 0 parameters = {:4.1f}%({}/{})'.
         format(near_zero_count/total_count*100, near_zero_count, total_count))
    
def cnn_reg_collect_params(self):
    params = list(self.pm_output['w'].flatten())
    for pm in self.pm_hiddens:
        if 'w' in pm: params += list(pm['w'].flatten())
        if 'k' in pm: params += list(pm['k'].flatten())
    return params

CnnRegModel.show_param_dist = cnn_reg_show_param_dist
CnnRegModel.collect_params = cnn_reg_collect_params

## 8.8.4 추가 손실 함수 계산 매서드 재정의

In [9]:
def cnn_reg_forward_extra_cost(self, y):
    extra, aux_extra = super(CnnRegModel, self).forward_extra_cost(y)
    if self.l2_decay > 0 or self.l1_decay > 0:
        params = self.collect_params()
        if self.l2_decay > 0:
            extra += np.sum(np.square(params)) / 2
        if self.l1_decay > 0:
            extra += np.sum(np.abs(params))
    return extra, aux_extra
                                            # 평가 단계에서는 불필요하지만 이 메서드는 손실 함수 계산 과정에서 호출되며 손실 함수,
                                            # 계산 과정은 원래 학습 단계에서만 호출되므로 self.is_training 플래그 값 검사는 불필요함
CnnRegModel.forward_extra_cost = cnn_reg_forward_extra_cost 

## 8.8.5 파라미터 갱신 함수 재정의

In [10]:
def cnn_reg_update_param(self, pm, key, delta):
    if self.use_adam:
        delta = self.eval_adam_delta(pm, key, delta)

    if key in ['w', 'k']:
        if self.l2_decay > 0:
            delta += self.l2_decay * pm[key]
        if self.l1_decay > 0:
            delta += self.l1_decay * np.sign(pm[key])
            
    pm[key] -= self.learning_rate * delta

CnnRegModel.update_param = cnn_reg_update_param

## 8.8.6 드롭아웃 계층 지원을 위한 세 가지 메서드 정의

In [11]:
def cnn_reg_alloc_dropout_layer(self, input_shape, hconfig):
    keep_prob = get_conf_param(hconfig, 'keep_prob', 1.0)
    assert 0 < keep_prob <= 1
    return {'keep_prob': keep_prob}, input_shape

def cnn_reg_forward_dropout_layer(self, x, hconfig, pm):
    if self.is_training:
        dmask = np.random.binomial(1, pm['keep_prob'], x.shape) # x.shape[1:]로 미니배치 단위로 일괄처리해야 효과적이라함(?)
        dropped = x * dmask / pm['keep_prob'] # 기댓값을 유지시키기 위해 남은 파라미터를 pm['keep_prob']의 역수만큼 증폭시킴
        return dropped, dmask
    else:
        return x, None
def cnn_reg_backprop_dropout_layer(self, G_y, hconfig, pm, aux):
    dmask = aux
    G_hidden = G_y * dmask / pm['keep_prob'] # 마스크를 곱해 사라진(0이된) 값을 곱해 0으로 만듬, 순전파와 마찬가지로 기울기 값도 증폭시킴
    return G_hidden

CnnRegModel.alloc_dropout_layer = cnn_reg_alloc_dropout_layer
CnnRegModel.forward_dropout_layer = cnn_reg_forward_dropout_layer
CnnRegModel.backprop_dropout_layer = cnn_reg_backprop_dropout_layer

## 8.8.7 잡음 주입 계층 지원을 위한 세 가지 메서드 정의

In [12]:
def cnn_reg_alloc_noise_layer(self, input_shape, hconfig):
    noise_type = get_conf_param(hconfig, 'type', 'normal')
    mean = get_conf_param(hconfig, 'mean', 0)
    std = get_conf_param(hconfig, 'std', 1.0)
    ratio = get_conf_param(hconfig, 'ratio', 1.0) # 잡음 주입 비율
    
    assert noise_type == 'normal' # 여기에서는 정규분포로 제한
    
    return {'mean':mean, 'std':std, 'ratio':ratio}, input_shape

def cnn_reg_forward_noise_layer(self, x, hconfig, pm):
    if self.is_training and np.random.rand() < pm['ratio']:
        noise = np.random.normal(pm['mean'], pm['std'], x.shape)
        return x + noise, None
    else:
        return x, None
    
def cnn_reg_backprop_noise_layer(self, G_y, hconfig, pm, aux):
    return G_y # 아무 일도 할 필요 없이 메서드 자체만 정의

CnnRegModel.alloc_noise_layer = cnn_reg_alloc_noise_layer
CnnRegModel.forward_noise_layer = cnn_reg_forward_noise_layer
CnnRegModel.backprop_noise_layer = cnn_reg_backprop_noise_layer

## 8.8.8 배치 정규화 계층 지원을 위한 세 가지 메서드 정의

In [22]:
def cnn_reg_alloc_batch_normal_layer(self, input_shape, hconfig):
    pm = {}
    rescale = get_conf_param(hconfig, 'rescale', True)
    pm['epsilon'] = get_conf_param(hconfig, 'epsilon', 1e-10)
    pm['exp_ratio'] = get_conf_param(hconfig, 'exp_ratio', 0.001)
    
    bn_dim = input_shape[-1] # 배치 정규화 그룹 수. 완전 연결 계층의 경우 미니배치 데이터의 입력 벡터 크기, 합성곱 계층의 경우 출력 채널수
    pm['mavg'] = np.zeros(bn_dim)
    pm['mvar'] = np.ones(bn_dim)
    if rescale: # 이 두 가지는 학습 대상 파라미터
        pm['scale'] = np.ones(bn_dim) 
        pm['shift'] = np.zeros(bn_dim)
    return pm, input_shape

def cnn_reg_forward_batch_normal_layer(self, x, hconfig, pm):
    if self.is_training:
        x_flat = x.reshape([-1, x.shape[-1]]) # 마지막 차원을 별로 일렬로 폄
        avg = np.mean(x_flat, axis=0)
        var = np.var(x_flat, axis=0)
        pm['mavg'] += pm['exp_ratio'] * (avg - pm['mavg']) # 평균의 이동 평균을 구함
        pm['mvar'] += pm['exp_ratio'] * (var - pm['mvar']) # 분산의 이동 평균을 구함
    else:                             # 구해놓은 이동 평균을 사용함
        avg = pm['mavg'] 
        var = pm['mvar'] 
    std = np.sqrt(var + pm['epsilon']) # 0으로 나누기 방지
    y = norm_x = (x - avg) / std
    if 'scale' in pm:
        y = pm['scale'] * norm_x + pm['shift'] 
    return y, [norm_x, std] #역전파 보조 정보로 norm_x 필요

def cnn_reg_backprop_batch_normal_layer(self, G_y, hconfig, pm, aux):
    norm_x, std = aux
    if 'scale' in pm: # n * m  m* k n *k
        if len(G_y.shape) == 2: axis = 0 # MLP(2차원)의 경우 미니배치 데이터 내에서 더함
        else: axis = (0,1,2) # CNN(4차원)의 경우 채널을 제외한 앞의 3차원 별로 모두 더함
        G_scale = np.sum(G_y*norm_x, axis = axis)
        G_shift = np.sum(G_y, axis=axis)
        G_y = G_y * pm['scale'] 
        pm['scale'] -= self.learning_rate * G_scale
        pm['shift'] -= self.learning_rate * G_shift
    G_input = G_y / std # 순전파 처럼 std로 나눠줌
    return G_input

CnnRegModel.alloc_batch_normal_layer = cnn_reg_alloc_batch_normal_layer
CnnRegModel.forward_batch_normal_layer = cnn_reg_forward_batch_normal_layer
CnnRegModel.backprop_batch_normal_layer = cnn_reg_backprop_batch_normal_layer